**Passage à une fonction: nettoyage et création d'une dataframe contenant le numéro de dossier et l'annotation**

à noter: ca ne marche pas avec les filehtml.ann mais avec les filepdf.ann


In [69]:
import pandas as pd
def transformation(numeroDossier, dossier):
    df= pd.read_csv(dossier, sep='\t')
    df.columns=['numéro','annotation','detail']
    
    # on transforme les annotations qui sont de type object et précisent
    #  la place dans le dossier du texte annoté en string 
    df_ann=df['annotation'].convert_dtypes(convert_string=True)
    df['annotation']=df_ann.str.split(expand=True)[0]
    
    # on supprime la colonne numéro
    df.pop('numéro')
    
    # on pivote la data frame pour pouvoir trier chaque détail par annotation
    df=df.pivot(columns='annotation')

    # on tej le multiindex détail qui sert à rien
    df=df[('detail')]

    # on ajoute une colonne numéro de doss
    df['Numéro Dossier']= numeroDossier
    
    # on crée des dataframes indépendantes pour les annotations qui nous intéressent:
    # traitement, pathologie et examen
    df_trait=df[['Numéro Dossier','traitement']].dropna()
    df_path=df[['Numéro Dossier','pathologie']].dropna()
    df_exam=df[['Numéro Dossier','examen']].dropna()
    print(df_path)
    print(df_trait)
    
    

In [70]:
# exemple pour 1 fichier:
transformation(numeroDossier=1,dossier='filepdf-5-cas.ann')

annotation  Numéro Dossier                         pathologie
2                        1  métastase hépatique et pulmonaire
3                        1                métastases cutanées
11                       1                       tumeur noire
16                       1                métastases cutanées
17                       1                       état général
18                       1              état général conservé
19                       1                     mélanome malin
27                       1            métastase ganglionnaire
28                       1                             tumeur
41                       1       adénopathie inguinale gauche
46                       1                             tumeur
49                       1           tumeur circonférentielle
57                       1                     mélanome malin
59                       1              métastases viscérales
annotation  Numéro Dossier            traitement
0                    

In [71]:
transformation(numeroDossier=2,dossier='filepdf-17-cas.ann')

annotation  Numéro Dossier                                     pathologie
1                        2                                      prolapsus
3                        2                         prolapsus de néo-vagin
21                       2              carcinome urothélial de l'urèthre
29                       2                                      métastase
33                       2  cancer de l'urèthre étendu au trigone vésical
45                       2                         prolapsus du néo-vagin
52                       2            adénopathie ilio-obturatrice gauche
annotation  Numéro Dossier                               traitement
15                       2                            voie vaginale
19                       2                anastomose caeco-vulvaire
22                       2                  reconstruction vaginale
32                       2                               anastomose
37                       2                                 incision


**On va maintenant créer une fonction qui prend plusieurs dossiers et renvoie une dataframe associée à chaque annotation**

In [133]:
# la fonction prend en argument une liste de dossiers patients

def creationDF(dossiers):
    pathologies=[]
    examens=[]
    traitements=pd.DataFrame()
    for i in range(len(dossiers)):
        # dû à un problème de scope des variables de ma fonction transformation, 
        # je dois redéfinir cette même fonction dans celle-ci
        # à l'origine j'avais écrit:
        #transformation(numeroDossier=i+1, dossier=dossiers[i])
        #pathologies.append(df_path)
        #examens.append(df_exam)
        #traitements.append(df_trait)
        
        df= pd.read_csv(dossiers[i], sep='\t')
        df.columns=['numéro','annotation','detail']
    
        df_ann=df['annotation'].convert_dtypes(convert_string=True)
        df['annotation']=df_ann.str.split(expand=True)[0]
    
        df.pop('numéro')
    
        df=df.pivot(columns='annotation')

        df=df[('detail')]

        df['Numéro Dossier']= i+1
        
        # on supprime la colonne annotation pcq osef. En fait ca marche pas donc va garder lol
        #df.drop('annotation', axis=1, inplace=True)
        
        df_trait=df[['Numéro Dossier','traitement']].dropna()
        df_path=df[['Numéro Dossier','pathologie']].dropna()
        df_exam=df[['Numéro Dossier','examen']].dropna()
        
        #pathologies.append(df_path)
        #examens.append(df_exam)
        traitements= traitements.append(df_trait, ignore_index=True)
        
    #juste pour avoir une visu de la data frame:
    print(traitements)
    #print(pathologies)
    #print(examens)
        
        

In [134]:
#test:
liste_dossiers=['filepdf-5-cas.ann','filepdf-17-cas.ann','filepdf-21-cas.ann']
premier_essai=creationDF(liste_dossiers)
print(premier_essai)
#note: je comprend pas pourquoi annotation prend alors un ordre alphabetique

annotation  Numéro Dossier                               traitement
0                        1                     pénectomie partielle
1                        1                               amputation
2                        2                            voie vaginale
3                        2                anastomose caeco-vulvaire
4                        2                  reconstruction vaginale
5                        2                               anastomose
6                        2                                 incision
7                        2                                   opérée
8                        2                  reconstruction vaginale
9                        2                                résection
10                       2  libération ascendante du caecum abaissé
11                       2                                   curage
12                       2                                   opérée
13                       2                   ana

** Premiers essais de visu **

In [103]:
import plotly as plt
import dash

In [132]:
premier_essai.shape()
#on a un probleme de tyoe.. il semblerait que la df soit passée en type NoneType

AttributeError: 'NoneType' object has no attribute 'shape'